## CS/DVR: step 1 and 2

In [1]:
import numpy as np
from numpy.linalg import eigvals
from scipy.optimize import minimize
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
%matplotlib qt5
import pandas as pd

In [2]:
#
#  extend path by location of the dvr package
#
import sys
sys.path.append('../../Python_libs')
import dvr
from jolanta import Jolanta_3D
from captools import simple_traj_der, five_point_derivative

In [3]:
amu_to_au=1822.888486192
au2cm=219474.63068
au2eV=27.211386027
Angs2Bohr=1.8897259886

In [4]:
#
# Jolanata-3D parameters a, b, c: (0.028, 1.0, 0.028)
#
#   CS-DVR:   
#      bound state: -7.17051 eV
#      resonance (3.1729556 - 0.16085j) eV
#
jparam=(0.028, 1.0, 0.028)

In [5]:
#
#  compute DVR of T and V
#  then show the density of states
#  in a potential + energy-levels plot
#
xmax=25
xmin=0   # grid from -xmin to xmax
thresh = 7   # maximum energy for state in the plot in eV
ppB = 15     # grid points per Bohr

nGrid=int((xmax-xmin)*ppB)
xs = dvr.DVRGrid(xmin, xmax, nGrid)
Ts = dvr.KineticEnergy(1, xmin, xmax, nGrid)
Vs=Jolanta_3D(xs, jparam)
[energy, wf] = dvr.DVRDiag2(nGrid, Ts, Vs, wf=True)

n_ene=0
for i in range(nGrid):
    print("%3d  %12.8f au = %12.5f eV" % (i+1, energy[i], energy[i]*au2eV))
    n_ene += 1
    if energy[i]*au2eV > thresh:
        break

# "DVR normalization", sum(wf[:,0]**2)
# this is correct for plotting

c=["orange", "blue"]
h=float(xmax) / (nGrid+1.0)
scale=400

plt.plot(xs,Vs*au2eV, '-', color="black")
for i in range(n_ene):
    plt.plot(xs, scale*wf[:,i]**2+energy[i]*au2eV, '-', color=c[i%len(c)])
#plt.ylim(0, 20)
plt.xlabel('$x$ [Bohr]')
plt.ylabel('$E$ [eV]')
plt.ylim(-8,8)
plt.show()

  1   -0.26351131 au =     -7.17051 eV
  2    0.02918102 au =      0.79406 eV
  3    0.09074913 au =      2.46941 eV
  4    0.12080260 au =      3.28721 eV
  5    0.18417311 au =      5.01161 eV
  6    0.27654358 au =      7.52513 eV


Two helper functions

In [6]:
def ECS(alpha, theta):
    """ diag H(alpha*exp(i*theta)) and return eigenvalues """
    eta=alpha*np.exp(1j*theta)
    Vs = Jolanta_3D(xs*eta, jparam)
    H_theta = Ts/eta**2 + np.diag(Vs)
    return eigvals(H_theta)

def near_E(alpha, theta, E_near):
    """ diag H(alpha*exp(i*theta)) and return eigenvalue near E_near """
    Es = ECS(alpha, theta)
    j = np.argmin(np.abs(Es - E_near))
    return Es[j]

In [7]:
""" complex diagonalization example """

theta=25.0/180.0*np.pi
energies = ECS(1.0, theta)
energies.sort()
energies *= au2eV
for i in range(10):
    print(i, energies[i])
plt.cla()
plt.plot(energies.real, energies.imag, 'o')
plt.xlim(-8,10)
plt.ylim(-2,0.5)
plt.show()

0 (-7.17048676323494-2.757442235297604e-05j)
1 (0.12319575418222453-0.6447856747464203j)
2 (0.6769051073757529-2.510872946805203j)
3 (2.2813269896572237-5.036498142249488j)
4 (3.1729350979582995-0.16086331778407406j)
5 (4.8475998455820974-8.113277014269515j)
6 (7.513829958365812-4.717125131061044j)
7 (8.121591146545715-11.38052093763252j)
8 (11.32008035953136-13.287205284627333j)
9 (13.083027798587672-16.59321807868231j)


For small $\theta$ everything is fine, but for larger angles basis set artifacts become apparent = there are eigenvalues off the rotation string with small real and huge imaginary part. Thus, above some $\theta$, sorting the eigenvalue by low real part stops working as a filter. 

In [8]:
#
#  complex scaling loop: 
#
#  start on the real axis (theta=0) and rotate to theta = theta_max 
#
#  we keep n_keep energies with the lowest real part 
#

n_keep=30
theta_min=0
theta_max=40
n_theta=theta_max - theta_min + 1
thetas=[t for t in range(theta_min, theta_max+1)]

run_data = np.zeros((n_theta,n_keep), complex)  # array used to collect all theta-run data
#run_data = np.zeros((n_keep,n_theta), complex)  # array used to collect all theta-run data

for i_theta in range(n_theta):
    theta=thetas[i_theta]/180.0*np.pi
    energies = ECS(1.0, theta)
    energies.sort()
    run_data[i_theta,:] = energies[0:n_keep]
    #run_data[:,i_theta] = energies[0:n_keep]
    print(i_theta+1, end=" ")
    if (i_theta+1)%10==0:
        print()

run_data *= au2eV

1 2 3 4 5 6 7 8 9 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 33 34 35 36 37 38 39 40 
41 

Plot the raw data; use matplotlib to zoom.

In [9]:
plt.cla()
for i in range(0, n_keep):
    plt.plot(run_data[:,i].real,  run_data[:,i].imag, 'o')
plt.xlim(-1,10)
plt.ylim(-1,0)
plt.show()

Two follow ideas:

1. at the last $\theta$ compute the angles and compare with $2*\theta$ if significantly smaller, then resonance
2.  once a trajectory has five values, use them to establish a Pade[2,2] then predict for the next theta



In [10]:
def follow_nearest(follow, es):
    """
    follow the energy closet to e0 from the real axis into the complex plane
    es is a table of theta-run data es[i_theta,j_energies]
    the algorithm used is simply nearest to the old energy
    """
    (n_thetas, n_energies) = es.shape
    trajectory = np.zeros(n_thetas,complex)
    for j in range(0,n_thetas):
        i = np.argmin(abs(es[j,:]-follow))
        follow = trajectory[j] = es[j,i]
    return trajectory

In [11]:
n_save = n_keep//2
trajectories = np.zeros((n_theta, n_save), complex)
for j in range(n_save):
    trajectories[:,j] = follow_nearest(run_data[0,j], run_data)
plt.xlim(-1,20)
plt.ylim(-10,0)
for i in range(0, n_save):
    plt.plot(trajectories[:,i].real,  trajectories[:,i].imag, '-')
plt.show()

In [12]:
#
#  save n_save trajectories to file
#  csv as real and imag 
#  (at the moment easier than csv with complex)
#  also, include no header, because the energies need to be sorted
#  into trajectories first
#
#fname="complex_scaling_rmax."+str(int(rmax))+"_ppB."+str(ppB)+".csv"
#read_write.write_theta_run(fname,thetas,trajectories)

The resonance clearly stands out in CS/DVR.
Follow the trajectory that finishes with the smallest angle.

In [13]:
for i in range(n_save):
    print(i, np.angle(trajectories[-1,i],deg=True))
print("Minimum angle index:",np.argmax(np.angle(trajectories[-1,0:n_save])))

0 -179.99980264790037
1 -114.86693835412485
2 -122.97131985558339
3 -2.90201435259857
4 -101.3736921632065
5 -32.109685024747904
6 -104.59355491505079
7 -95.13707394048981
8 -52.29813433621948
9 -93.21931421284157
10 -63.17564494933142
11 -63.17564494933142
12 -87.11382163838118
13 -70.16399195953744
14 -87.11382163838118
Minimum angle index: 3


In [14]:
res_traj = trajectories[:,3]
abs_der = np.abs(simple_traj_der(thetas, res_traj))
plt.cla()
plt.plot(thetas, np.log(abs_der), 'o-')
plt.xlabel(r'$\theta$')
plt.ylabel(r'$\ln|dE/d\theta|$')
plt.show()

Energy, $\theta_{opt}$, and stability for $\alpha=1$

In [15]:
j_opt = np.argmin(abs_der)
if j_opt == len(thetas)-1:
    j_opt -= 1
theta_opt = thetas[j_opt]
Eres=res_traj[j_opt]
print('theta_opt', thetas[j_opt])
print('E_res', res_traj[j_opt])
print('dE/dtheta', abs_der[j_opt])
print('delta-E down', res_traj[j_opt-1]-res_traj[j_opt])
print('delta-E up  ', res_traj[j_opt+1]-res_traj[j_opt])

theta_opt 39
E_res (3.1729627262865874-0.16084768159778406j)
dE/dtheta 1.4402761214424823e-06
delta-E down (-1.4049190060205774e-06-3.1717170825840135e-07j)
delta-E up   (1.3641803864317126e-06+4.161990286022377e-07j)


## Optimize $\eta=\alpha\,\exp(i\theta)$

* Define the objective $\vert dE/dz\vert$ using average central derivatives.
* Find its minimum using simplex minimization.

However, even though $\eta$ changes quite a bit and $\vert dE/dz\vert$ decreases quite a lot, $E_{res}$ is essentially constant.

In [27]:
def objective(eta, dx, E_near, verbose=False):
    """
    Input: 
        eta: (alpha, theta); z = alpha*exp(i*theta)
        dx:  numerical derivatives using z +/- dx or idx
        E_near: select the eigenvalue nearest to this energy  
    Output: 
        dE/dz along Re(z), dE/dz along Im(z) for the state near E_near
    This requires four single points.
    """
    a, t = eta
    z0 = a*np.exp(1j*t)
    es = np.zeros(2, complex)
    zs = np.array([z0-dx, z0+dx])
    for i, z in enumerate(zs):
        a, t = np.abs(z), np.angle(z)
        es[i] = near_E(a, t, E_near)
    dEdz_re = (es[1] - es[0])/(2*dx)
    zs = [z0-1j*dx, z0+1j*dx]
    for i, z in enumerate(zs):
        a, t = np.abs(z), np.angle(z)
        es[i] = near_E(a, t, E_near)
    dEdz_im = (es[1] - es[0])/(2j*dx)
    if verbose:
        print(dEdz_re)
        print(dEdz_im)
    return np.abs(0.5*(dEdz_re + dEdz_im)) 

In [29]:
alpha_curr=1.0
theta_curr=theta_opt/180.0*np.pi
dx=0.001
E_find = Eres/au2eV
objective((alpha_curr, theta_curr), dx, E_find, verbose=True)

(-1.229734536423166e-06-2.746350224059474e-06j)
(-1.2323924459059499e-06-2.749129306145459e-06j)


3.0109120106711237e-06

For DVR, finding $\eta_{opt}=\alpha_{opt}\exp(i\theta_{opt})$ *via* minimization is very slow. Computing $E_{res}$ on a grid and interpolating should be much faster.

In [30]:
args=(dx, E_find)
p0=[alpha_curr, theta_curr]
print('p0=', p0)
print('Eres0=', near_E(alpha_curr, theta_curr, E_find)*au2eV)
print('f0=', objective(p0, dx, E_find))
res=minimize(objective, p0, args=args, method="Nelder-Mead")
print(res.message)
print(res.x)
print(res.fun)
Eopt=near_E(res.x[0], res.x[1], E_find)*au2eV
print('Eres=', Eopt)
print('Energy change=', Eopt-Eres)

p0= [1.0, 0.6806784082777886]
Eres0= (3.1729627262865874-0.16084768159778406j)
f0= 3.0109120106711237e-06
Optimization terminated successfully.
[0.9415283  0.70868483]
8.656724926280831e-09
Eres= (3.1729630104542235-0.16084352057881723j)
Energy change= (2.8416763608873907e-07+4.161018966830632e-06j)
